In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from hdfs import InsecureClient
from scipy.stats import skew 
import os
import mysql.connector as mariadb

In [4]:
client_hdfs = InsecureClient('http://10.1.41.102:50070')

In [22]:
def total_data():
    raw_file_list = ['pre_dataset_bakery.csv', 'pre_dataset_cafe.csv', 'pre_dataset_chicken.csv', 'pre_dataset_chinese.csv',
            'pre_dataset_fastFood.csv', 'pre_dataset_hof.csv', 'pre_dataset_japanese.csv', 'pre_dataset_korean.csv',
            'pre_dataset_snack.csv', 'pre_dataset_western.csv']

    df = pd.DataFrame(columns=['date', 'areaCode', 'serviceCode', 'totalNearStore', 'totalStore', 'avgNearMonth', 'nearSales', 'numberOfNearSales', 'avgMonth',
           'numberOfSales', 'totalPeople', 'totalNearPeople', 'totalBizman','totalNearBizman', 'totalLivingPeople', 'avgIncome', 'avgOutcome',
           'avgNearIcome', 'avgNearOutcome', 'totalFacility', 'totalNearFacility', 'sales'])

    for csv in raw_file_list:
        with client_hdfs.read('/user/hadoop/conf/'+csv, encoding = 'utf-8') as reader:
            df_read = pd.read_csv(reader)
        df = pd.concat([df, df_read])
    df= df[['date', 'areaCode', 'serviceCode', 'totalNearStore', 'totalStore',
       'avgNearMonth', 'nearSales', 'numberOfNearSales', 'avgMonth',
       'numberOfSales', 'totalPeople', 'totalNearPeople', 'totalBizman',
       'totalNearBizman', 'totalLivingPeople', 'avgIncome', 'avgOutcome',
       'avgNearIcome', 'avgNearOutcome','totalFacility', 'totalNearFacility', 'sales']]
    return df

In [23]:
latest_data = total_data()

In [24]:
raw_data = latest_data[['date', 'areaCode', 'serviceCode', 'totalNearStore', 'totalStore',
       'avgNearMonth', 'nearSales', 'numberOfNearSales', 'avgMonth',
       'numberOfSales', 'totalPeople', 'totalNearPeople', 'totalBizman',
       'totalNearBizman', 'totalLivingPeople', 'avgIncome', 'avgOutcome',
       'avgNearIcome', 'totalFacility', 'totalNearFacility', 'sales']]

meaning_data = raw_data.iloc[:,3:]
#object타입을 float로 변환
col_goto_change = meaning_data.dtypes[meaning_data.dtypes == "object"].index
meaning_data[col_goto_change] = meaning_data[col_goto_change].apply(lambda x :x.astype('float64', raise_on_error = False))

# #쓸 자료들 중에 왜도를 na로 바꿈
skewness = meaning_data.apply(lambda x : skew(x.dropna()))

#뽑힌 feature를 log1p 적용
meaning_data = np.log1p(meaning_data)
#변수가 헷갈릴 수 있으니 logated_data로 명시
logated_data = meaning_data
#이상치 제거된 na값들을 평균값으로 대체, 소수점 2자리까지 반영
logated_data = logated_data.fillna(logated_data.mean()).apply(lambda x : round(x, 2))
#return logated_data
#logated_data.fillna(0.0)
X_origin = logated_data.drop(['sales'], axis=1)
Y = logated_data['sales']
scaler = MinMaxScaler()
X = scaler.fit_transform(X_origin)

/home/el/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: the raise_on_error=False keyword is deprecated, use errors='ignore' instead
/home/el/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in log1p


In [25]:
raw_data.iloc[:,3:-1] = X
raw_data['sales'] = Y

/home/el/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/el/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [31]:
df = raw_data[raw_data['date']==201708.0]

In [69]:
# def get_feature_in_latest_data(latest_data):
#     meaning_data = latest_data.iloc[:,2:]
#     #object타입을 float로 변환
#     col_goto_change = meaning_data.dtypes[meaning_data.dtypes == "object"].index
#     meaning_data[col_goto_change] = meaning_data[col_goto_change].apply(lambda x :x.astype('float64', raise_on_error = False))
#     #쓸 자료들 중에 왜도를 na로 바꿈
#     skewness = meaning_data.apply(lambda x : skew(x.dropna()))
#     # skewness가 0.75이상인 값들만 feature로 사용
#     skewed_feats = skewness[skewness>0.75].index
#     #뽑힌 feature를 log1p 적용
#     meaning_data[skewed_feats] = np.log1p(meaning_data[skewed_feats])
#     #변수가 헷갈릴 수 있으니 logated_data로 명시
#     logated_data = meaning_data[skewed_feats]
#     #이상치 제거된 na값들을 평균값으로 대체
#     logated_data = logated_data.fillna(logated_data.mean())
#     X_origin = logated_data.drop(['sales'],axis=1)
#     scaler = MinMaxScaler()
#     X = scaler.fit_transform(X_origin)
#     logated_data.iloc[:,:-1] = X
#     latest_data.iloc[:,2:] = logated_data
#     df = latest_data.dropna(axis=1)
#     df.to_csv("../output/total_latest_data.csv", index=False)
#     return df

In [40]:
df
df.to_csv("../output/total_latest_data.csv", index=False)

In [36]:
df.columns

Index(['date', 'areaCode', 'serviceCode', 'totalNearStore', 'totalStore',
       'avgNearMonth', 'nearSales', 'numberOfNearSales', 'avgMonth',
       'numberOfSales', 'totalPeople', 'totalNearPeople', 'totalBizman',
       'totalNearBizman', 'totalLivingPeople', 'avgIncome', 'avgOutcome',
       'avgNearIcome', 'totalFacility', 'totalNearFacility', 'sales'],
      dtype='object')

In [37]:
mariadb_connection = mariadb.connect(user='root', password='el', database='el')
cursor = mariadb_connection.cursor()

In [38]:
for row in df.iterrows():
    input_value =row[1]
    cursor.execute("insert into 1708_data values ({})".format(",".join(repr(e) for e in input_value)))

In [39]:
cursor.execute("commit")